<a href="https://colab.research.google.com/github/anassboussarhan/tfrecords/blob/master/tfrecords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import random
import sys
import threading
import numpy as np
import tensorflow as tf
from datetime import datetime
from queue import Queue
from glob import glob

image_dir='/content/drive/My Drive/Projetand/train/image/'
mask_dir='/content/drive/My Drive/Projetand/train/mask/'
image_suffix=".png"
mask_suffix=".png"
train_size=1
validation_size=0
output_dir='/content/drive/My Drive/Projetand/'
shards=1
threads=1
shuffle=False


  

In [0]:
from google.colab import drive
drive.mount('/content/drive/')




Mounted at /content/drive/


In [0]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _convert_to_example(image_buffer, mask_buffer, filename, mask_filename):
    example = tf.train.Example(features=tf.train.Features(feature={
        'image': _bytes_feature(image_buffer),
        'mask': _bytes_feature(mask_buffer),
    }))
    return example


def _process_image_files_batch(thread_index, batch_data, shards, total_shards, mask_dir, mask_suffix, output_dir,
                               output_name, error_queue):
    batch_size = len(batch_data)
    batch_per_shard = batch_size // len(shards)

    counter = 0
    error_counter = 0
    for s in range(len(shards)):
        shard = shards[s]
        output_filename = '%s-%.5d-of-%.5d' % (output_name, shard, total_shards)
        output_file = os.path.join(output_dir, output_filename)

        writer = tf.python_io.TFRecordWriter(output_file)
        shard_counter = 0
        shard_range = (s * batch_per_shard, min(batch_per_shard, batch_size - (s * batch_per_shard)))
        files_in_shard = np.arange(shard_range[0], shard_range[1], dtype=int)
        for i in files_in_shard:
            filename = data[i]
            mask_filename = os.path.join(mask_dir,
                                         os.path.splitext(os.path.split(filename)[-1])[0] + mask_suffix)
            try:
                image_buffer = tf.gfile.FastGFile(filename, 'rb').read()
                mask_buffer = tf.gfile.FastGFile(mask_filename, 'rb').read()
                example = _convert_to_example(image_buffer, mask_buffer, filename, mask_filename)
                writer.write(example.SerializeToString())
                shard_counter += 1
                counter += 1
            except StopIteration as e:
                error_counter += 1
                error_msg = repr(e)
                error_queue.put(error_msg)

        print('%s [thread %d]: Wrote %d images to %s, with %d errors.' %
              (datetime.now(), thread_index, shard_counter, output_file, error_counter))
        sys.stdout.flush()

    print('%s [thread %d]: Wrote %d images to %d shards, with %d errors.' %
          (datetime.now(), thread_index, counter, len(shards), error_counter))


def create(data, mask_dir, mask_suffix, output_name, output_dir, num_shards, num_threads):
    num_data_per_thread = len(data) // num_threads
    num_shard_per_thread = num_shards // num_threads
    batch_data_ranges = [(i * num_data_per_thread, min(num_data_per_thread, len(data) - i * num_data_per_thread))
                         for i in range(num_threads)]

    coord = tf.train.Coordinator()

    error_queue = Queue()

    threads = []
    for thread_index in range(1, num_threads + 1):
        batch_ranges = batch_data_ranges[thread_index - 1]
        batch_data = data[batch_ranges[0]:batch_ranges[1]]
        shards = [thread_index + (thread_index - 1) * (num_shard_per_thread - 1) + shard
                  for shard in range(num_shard_per_thread)]
        args = (thread_index, batch_data, shards, num_shards, mask_dir, mask_suffix, output_name, output_dir,
                error_queue)
        t = threading.Thread(target=_process_image_files_batch, args=args)
        t.start()
        threads.append(t)

    coord.join(threads)

    errors = []
    while not error_queue.empty():
        errors.append(error_queue.get())
    print('%d examples failed.' % (len(errors),))






data = glob(os.path.join(image_dir, "*" + image_suffix))



num_data = len(data)
num_train = int(round(num_data * train_size))
num_validation = round(num_data * validation_size)

training = data[1:num_train]
create(training, mask_dir, mask_suffix, output_dir, "train", shards, threads)
if validation_size > 0:
    validation = data[num_train:num_train + num_validation]
    create(validation, mask_dir, mask_suffix, output_dir, "val", shards, threads)
if train_size + validation_size < 1:
    test = data[num_train + num_validation:]
    create(test, mask_dir, mask_suffix, output_dir, "test", shards, threads)


2018-10-13 01:40:22.724733 [thread 1]: Wrote 26 images to /content/drive/My Drive/Projetand/train-00001-of-00001, with 0 errors.
2018-10-13 01:40:22.726194 [thread 1]: Wrote 26 images to 1 shards, with 0 errors.
0 examples failed.
